In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [8]:
linear1 = torch.nn.Linear(784, 512, bias=True)
linear2 = torch.nn.Linear(512, 512, bias=True)
linear3 = torch.nn.Linear(512, 512, bias=True)
linear4 = torch.nn.Linear(512, 512, bias=True)
linear5 = torch.nn.Linear(512, 10, bias=True)
relu = torch.nn.ReLU()

In [9]:
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[-0.0274, -0.0604, -0.0527,  ..., -0.0496,  0.0994,  0.0173],
        [ 0.0431,  0.0969, -0.0755,  ..., -0.0875, -0.0003,  0.0079],
        [ 0.1007, -0.0730,  0.0812,  ..., -0.0994, -0.0020, -0.0903],
        ...,
        [-0.0977, -0.0285,  0.0882,  ..., -0.0371, -0.0694, -0.0768],
        [ 0.0226,  0.0896,  0.0393,  ...,  0.0607, -0.0032, -0.0567],
        [ 0.0854, -0.0383, -0.0931,  ..., -0.0531,  0.0711,  0.0584]],
       requires_grad=True)

In [10]:
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

In [11]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y, in data_loader:
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.281641603
Epoch: 0002 cost = 0.089465454
Epoch: 0003 cost = 0.057418730
Epoch: 0004 cost = 0.039944962
Epoch: 0005 cost = 0.031628460
Epoch: 0006 cost = 0.023338282
Epoch: 0007 cost = 0.020518446
Epoch: 0008 cost = 0.019795565
Epoch: 0009 cost = 0.014289372
Epoch: 0010 cost = 0.013702988
Epoch: 0011 cost = 0.015635595
Epoch: 0012 cost = 0.010528967
Epoch: 0013 cost = 0.010265669
Epoch: 0014 cost = 0.010486320
Epoch: 0015 cost = 0.011031301
Learning finished


In [13]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())
    
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r+1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r+1].to(device)
    
    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.979200005531311
Label:  8
Prediction:  8


/opt/conda/lib/python3.6/site-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/opt/conda/lib/python3.6/site-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
